In this code file, we use the following files/directories to create the variables used for our regressions:
1. All RTO dataframes (2010-23), a directory which contains excel files of monthy RTO level data for more than 1200 RTOs for the time period Jan 2010 - Dec 2023 (link: https://vahan.parivahan.gov.in/vahan4dashboard/vahan/view/reportview.xhtml).

2. Updated arcGIS RTO to district matching.xlsx, an excel file which contains the mapping of RTOs to their present districts (link: https://www.arcgis.com/home/item.html?id=79875d04f49241979bed4b7a8e5bfdd8#data). In cases where the website does not contain a particular RTO to district mapping, I map these manually.

3. final_mapping.csv, a csv file which maps the arcGIS reported districts to their 2011 counterparts. This is done for consistency purposes and is also done manually. In cases where current districts come from multiple 2011 districts, they will be mapped to the group of districts

4. 2011 districts datameet boundaries.zip, a directory which contains a shapefile of the boundaries for districts according to the 2011 census (link: https://projects.datameet.org/maps/districts/). Based on final_mapping.csv, I create new district boundaries (including for group of districts) in latest grouped district boundaries.shp.

5. correct district naming.xlsx, an excel file which contains the names of 2011 districts as present in government documents. Again, done for consistency purposes and in case demographic data is to be incorporated later.

6. weather data, a directory which contains excel files of multiple weather readings for coordinates across India.

7. India urbanness shapefile Columbia.zip, a file which contains the boundaries of areas on their urbanness classification, based on 2011 factors. Urbanness is defined according to GHSL (link: https://sedac.ciesin.columbia.edu/data/set/india-spatial-india-census-2011/data-download).

8. higher resolution PM2.5 data monthly files, a directory containing netCDF files with monthly PM2.5 data at coordinates across India (link: https://sites.wustl.edu/acag/datasets/surface-pm2-5/). We use version V5.GL.01.

9. Monthly nightlight data (all years), a directory containing satellite images of VIIRS nightlight data across the world. This is at a monthly frequency (link: https://eogdata.mines.edu/nighttime_light/monthly/v10/). Available for April 2012 onwards.

As we progress, one thing to keep in mind is the datetime format in python. Let's say a row corresponds to November, 2023. To save it in a datetime object format, it gets saved as 2023-11-01 by default. This does not mean however that the row was recorded on November 1, 2023. In the final section, when we are merging our datasets, we will be dealing with aggregates (total registrations, average PM2.5, average urban nightlight, etc.) and even though the row would mention the date YYYY-MM-01, it wouldn't mean it's for the first day but instead for the entire month.

In [ ]:
# LIBRARIES 
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
import zipfile as zf
import os
import re
from collections import Counter
from netCDF4 import Dataset
import xarray as xr
import numpy as np
from sklearn.neighbors import NearestNeighbors
from netCDF4 import Dataset
from rasterio.enums import Resampling
import rioxarray as rxr
import rasterio

# Section 1: Mapping vehicle registration data to 2011 districts

In [ ]:
# DEFINING A FUNCTION TO EXTRACT RELEVANT VARIABLES FROM THE TABULAR FORMAT OF THE EXCEL FILES
def table_to_data(table):
    
    info = table.columns[0]
    
       
    # TAKING TAIL(-2) EXCLUDES THE TOP 2 NULL VALUES ROWS
    table = table.tail(-2)


    # CHANGING THE NAME OF THE COLUMN HEAD TO MAKE IT EASIER FOR NAME MANIPULATION LATER ON
    table.iloc[0][1] = table.columns[1]
    table.iloc[0][-1] = table.columns[-1]

    # REPLACING THE COLUMN NAMES
    table.columns = table.iloc[0] 
    table.iloc[0][-1] = table.columns[-1]

    table = table.rename(columns = {table.columns[0] : 'serial',
                                        table.columns[1] : 'fuel', table.columns[-1] : 'total'})
    table = table.drop(columns = ['serial', 'total'])

    # GETTING RID OF THE USELESS FIRST ROW
    table = table.reset_index(drop = True).loc[1:]

    # CONVERTS INTO A LONG FORMAT
    table = pd.melt(table, id_vars = ['fuel'],
           var_name = 'month', value_name = 'count')
    table['info'] = info
    
    return table

In [ ]:
# EXAMPLE OF WHAT THE EXCEL TABLES LOOK LIKE

registration_files = zf.ZipFile("All RTO dataframes (2010-23).zip", 'r')
registration_files.extractall('Vehicle registration files')

files_list = os.listdir('Vehicle registration files/All RTO dataframes (2010-23)')
example_table = pd.read_excel('Vehicle registration files/All RTO dataframes (2010-23)/' + files_list[0])
example_table

In [ ]:
# EXAMPLE OF WHAT THE table_to_data FUNCTION DOES WITH THE EXCEL TABLE AS INPUT
table_to_data(example_table)

In [ ]:
# APPLYING THE table_to_data FUNCTION TO EACH OF THE 17000 OR SO EXCEL FILES AND ADDING THEM ALL TO A LIST CALLED
# tables_list

files_list = os.listdir('Vehicle registration files/All RTO dataframes (2010-23)')
tables_list = []

for file_number in range(0,len(files_list)):
    if '__MACOSX' in files_list[file_number]:
        pass
    elif '.ipynb_checkpoints' in files_list[file_number]:
        pass
    
    else: 
        current_table = pd.read_excel('Vehicle registration files/All RTO dataframes (2010-23)/' + files_list[file_number])
        current_table = table_to_data(current_table)
        
        tables_list.append(current_table)

In [ ]:
# VERTICALLY APPENDING ALL THE TABLES FROM tables_list INTO ONE MAIN DATAFRAME CALLED main_df

main_df = pd.concat(tables_list, axis = 0).reset_index(drop = True)
main_df = main_df.drop_duplicates()

# CREATING ROWS FOR OTHER RELEVANT VARIABLES USING THE info COLUMN

# COLUMNS TO SEPERATE OUT THE INFORMATION 

main_df['state_year'] = main_df['info'].apply(lambda x: x.split('Fuel Month Wise Data  of')[1].split(',')[-1].strip())
main_df['year'] = main_df['info'].apply(lambda x: x.split('(')[-1][:-1].strip())
main_df['state'] = main_df['state_year'].apply(lambda x: x.split('(')[0].strip())
main_df['rto'] = main_df['info'].apply(lambda x: ''.join(x.split(',')[:-1]).split('Fuel Month Wise Data  of')[1].strip())
main_df['rto_code'] = main_df['rto'].apply(lambda x: x.split('-')[-1].strip())
main_df['rto_name'] = main_df['rto'].apply(lambda x: ' '.join(x.split('-')[:-1]).strip())
main_df['state_symbol'] = main_df['rto_code'].str.extract('([a-zA-Z]+)')
main_df = main_df.drop(columns = ['info', 'state_year'])

# AP126 DOES NOT HAVE VALUES FOR 2023 AND THEREFORE WE SKIP OUT THIS RTO FROM FURTHER ANALYSIS

main_df = main_df[main_df['rto_code'] != 'AP126']

# CONVERTING TO INTEGER VALUES FOR CALCULATION

main_df['count'] = main_df['count'].apply(lambda x: x.replace(',', ''))
main_df['count'] = main_df['count'].astype(int)

In [ ]:
main_df

In [ ]:
# CREATING 2 SEPERATE DATAFRAMES TO COUNT TOTAL MONTHLY REGISTRATIONS AT THE RTO LEVEL AND MONTHLY EV REGISTRATIONS
# AT THE RTO LEVEL

rto_month_counts = main_df.groupby(by = ['month', 'year', 'rto_name', 'state', 'state_symbol',
                                         'rto_code'])['count'].sum().to_frame().reset_index().rename(columns = 
                                                                                                     {'count' :
                                                                                                     'overall_count'})

ev_counts = main_df[main_df['fuel'] == 'ELECTRIC(BOV)'][['month', 'year', 'rto_name', 'state', 'state_symbol',
                                                'rto_code', 'count']]

ev_counts = ev_counts.rename(columns = {'count' : 'ev_count'})

# CREATING THE EV SHARES DATAFRAME

ev_shares = pd.merge(rto_month_counts, ev_counts, on = ['month', 'year', 'rto_name',
                                                        'rto_code', 'state', 'state_symbol'], how = 'outer')


# FOR MONTHS WITH NO REGISTRATIONS BUT TO STILL CALCULATE THE FRACTION

ev_shares['overall_count'] = ev_shares['overall_count'].replace(0, 1) 

ev_shares['ev_count'] = ev_shares['ev_count'].fillna(0).astype(int)
ev_shares['ev_share'] = ((ev_shares['ev_count']/ev_shares['overall_count'])*100).round(3)

In [ ]:
ev_shares

In [ ]:
# MAPPING EACH OF THE RTOS TO THEIR RESPECTIVE DISTRICTS, WHILE TAKING CARE OF SOME NAMING DISCREPANCIES

matching_districts = ev_shares[['rto_name']].drop_duplicates()
matching_districts['rto_name'] = matching_districts['rto_name'].apply(lambda x: x.strip())

updated_matching = pd.read_excel('Updated arcGIS RTO to district matching.xlsx')
updated_matching = updated_matching[['rto_name', 'rto_name_y', 'rto_code', 'district']]
full_match = pd.merge(matching_districts, updated_matching, on = 'rto_name')
for i in range(len(full_match)):
    if full_match['district'][i] == 'Angul':
        full_match['district'][i] = 'Anugul'
    elif full_match['district'][i] == 'Thoothukudi':
        full_match['district'][i] = 'Tuticorin'

In [ ]:
full_match

In [ ]:
# ADDING DISTRICT INFORMATION TO THE ev_shares DATAFRAME CREATED A FEW CELLS ABOVE.

# COMPARED TO EV_SHARES, WE LOSE 168 OBSERVATIONS AS WE DROP PUNJAB(STA) SINCE WE COULD NOT MATCH IT TO A DISTRICT
district_data = pd.merge(ev_shares, full_match, on = ['rto_name'])

district_data = district_data[['rto_name', 'rto_code_x', 'district', 'state', 'state_symbol',
               'year', 'month', 'overall_count', 'ev_count', 'ev_share']].rename(columns = 
                                                                                 {'rto_code_x' : 'rto_code'})

# CONTAINS THE CURRENT DISTRICT TO 2011 DISTRICT TO 2011 DISTRICT (IN DEMOGRAPHIC FILES) NAME CONVERSION

districts_2011 = pd.read_csv('final_mapping.csv').drop(columns = 'Unnamed: 0')

# RTOS MAPPED TO DISTRICTS

mapped_districts = pd.merge(full_match, districts_2011, left_on = ['district'],
         right_on = ['district_current (stage 1)'], how = 'left')

mapped_districts = mapped_districts[['rto_name', 'district','district_current (stage 1)',
                  'district_2011 (stage 2)']].drop_duplicates().reset_index(drop = True)

merged_df1 = pd.merge(district_data, mapped_districts, on = ['rto_name', 'district'])
merged_df1['district_2011 (stage 2)'] = merged_df1['district_2011 (stage 2)'] + ', ' + merged_df1['state_symbol']

In [ ]:
merged_df1

In [ ]:
# FURTHER RECONCILING NAMING DIFFERENCES ACROSS DIFFERENT DATAFRAMES

for i in range(len(merged_df1)):
    
    if merged_df1['district_2011 (stage 2)'][i] == 'Ahmedabad/Bhavnagar, GJ':
        merged_df1['district_2011 (stage 2)'][i] = 'Ahmadabad/Bhavnagar, GJ'
        
    elif merged_df1['district_2011 (stage 2)'][i] == 'Leh (Ladakh), LA':
        merged_df1['district_2011 (stage 2)'][i] = 'Leh (Ladakh), JK'
        merged_df1['state_symbol'][i] = 'JK'
        
    elif merged_df1['district_2011 (stage 2)'][i] == 'Kargil, LA':
        merged_df1['district_2011 (stage 2)'][i] = 'Kargil, JK'
        merged_df1['state_symbol'][i] = 'JK'    
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'Balrampur, CG':
        merged_df1['district_2011 (stage 2)'][i] = 'Surguja, CG'
        merged_df1['state_symbol'][i] = 'CG' 
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'East Godavari , AP':
        merged_df1['district_2011 (stage 2)'][i] = 'East Godavari, AP'
        merged_df1['state_symbol'][i] = 'AP' 
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'Panch Mahals, GJ':
        merged_df1['district_2011 (stage 2)'][i] = 'Panchmahal, GJ'
        merged_df1['state_symbol'][i] = 'GJ' 
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'Panchmahals, GJ':
        merged_df1['district_2011 (stage 2)'][i] = 'Panchmahal, GJ'
        merged_df1['state_symbol'][i] = 'GJ'
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'Panch mahal, GJ':
        merged_df1['district_2011 (stage 2)'][i] = 'Panchmahal, GJ'
        merged_df1['state_symbol'][i] = 'GJ'
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'Gurgaon , HR':
        merged_df1['district_2011 (stage 2)'][i] = 'Gurgaon, HR'
        merged_df1['state_symbol'][i] = 'HR'
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'Chandigarh, HR':
        merged_df1['district_2011 (stage 2)'][i] = 'Chandigarh, CH'
        merged_df1['state_symbol'][i] = 'CH'
        merged_df1['state'][i] = 'Chandigarh'
    
    elif merged_df1['district_2011 (stage 2)'][i] == 'Multiple, GJ':
        merged_df1['district_2011 (stage 2)'][i] = 'Surendranagar/Rajkot, GJ'
    
    
    else:
        pass

In [ ]:
# NOTICING THAT SOME RTOS BELONG TO DISTRICTS THAT ARE COMPOSED OF MULTIPLE 2011 DISTRICTS, WE NEED TO MAP THAT
# RTO'S FIGURES TO THE AMALGAMATION OF THE MULTIPLE DISTRICTS. IN OUR CONTEXT, THE MAXIMUM DISTRICTS THAT A SINGLE
# RTO BELONGS TO IS 2. HENCE, WE CREATE A SECOND COLUMN NAMED district_2011 (stage 2.1) TO KEEP TRACK OF THAT.

merged_df1['district_2011 (stage 2.1)'] = np.nan
for i in range(len(merged_df1)):
    if '/' in merged_df1['district_2011 (stage 2)'][i]:
        merged_df1['district_2011 (stage 2.1)'][i] = merged_df1['district_2011 (stage 2)'][i].split('/')[1]
    else:
        merged_df1['district_2011 (stage 2.1)'][i] = merged_df1['district_2011 (stage 2)'][i]

# leaves only 1 district in the first column
for i in range(len(merged_df1)):
    if '/' in merged_df1['district_2011 (stage 2)'][i]:
        merged_df1['district_2011 (stage 2)'][i] = merged_df1['district_2011 (stage 2)'][i].split('/')[0] + ', ' + merged_df1['state_symbol'][i]
    else:
        pass

    
# Function to get unique values from two columns in a row
def get_unique_values(row):
    return set([row['district_2011 (stage 2)'],row['district_2011 (stage 2.1)']])

# Apply the function to create a new column 'unique_districts'
merged_df1['unique_districts'] = merged_df1.apply(get_unique_values, axis=1)

# create the necessary groupings
groupings = []
for i in range(len(merged_df1)):
    if len(merged_df1['unique_districts'][i]) == 2:
        groupings.append(merged_df1['unique_districts'][i])

# function to get unique values
def unique_elements(list1):

# initialize a null list
    unique_list = []

# traverse for all elements
    for x in list1:
# check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

unique_elements(groupings)

In [ ]:
# CREATING THE GROUPINGS BASED ON THE OUTPUT OF THE unique_elements FUNCTION

merged_df1['grouped_district'] = np.nan
for i in range(len(merged_df1)):
    if merged_df1['district_2011 (stage 2)'][i] in {'Ahmadabad, GJ', 'Bhavnagar, GJ'}:
        merged_df1['grouped_district'][i] = 'Ahmadabad + Bhavnagar, GJ'
    
    elif merged_df1['district_2011 (stage 2)'][i] in {'Chittoor, AP', 'Y.S.R., AP'}:
        merged_df1['grouped_district'][i] = 'Chittoor + Y.S.R., AP'
    
    elif merged_df1['district_2011 (stage 2)'][i] in {'Faridabad, HR', 'Gurgaon, HR'}:
        merged_df1['grouped_district'][i] = 'Faridabad + Gurgaon, HR'
    
    elif merged_df1['district_2011 (stage 2)'][i] in {'Kheda, GJ', 'Panchmahal, GJ'}:
        merged_df1['grouped_district'][i] = 'Kheda + Panchmahal, GJ'
    
    elif merged_df1['district_2011 (stage 2)'][i] in {'Budaun, UP', 'Moradabad, UP'}:
        merged_df1['grouped_district'][i] = 'Budaun + Moradabad, UP'
    
    elif merged_df1['district_2011 (stage 2)'][i] in {'Rajkot, GJ', 'Surendranagar, GJ'}:
        merged_df1['grouped_district'][i] = 'Rajkot + Surendranagar, GJ'
    
    elif merged_df1['district_2011 (stage 2)'][i] in {'Rae Bareli, UP', 'Sultanpur, UP'}:
        merged_df1['grouped_district'][i] = 'Rae Bareli + Sultanpur, UP'
    
    elif merged_df1['district_2011 (stage 2)'][i] in {'Srikakulam, AP', 'Vizianagaram, AP'}:
        merged_df1['grouped_district'][i] = 'Srikakulam + Vizianagaram, AP'
    
    else:
        merged_df1['grouped_district'][i] = merged_df1['district_2011 (stage 2)'][i]

In [ ]:
# CREATING THE EV SHARES FOR EACH GROUPED DISTRICT
merged_df2 = merged_df1[['rto_name', 'rto_code', 'grouped_district', 'state', 'state_symbol',
           'year', 'month', 'overall_count', 'ev_count', 'ev_share']]

merged_df3 = merged_df2.groupby(by = ['grouped_district', 'year', 'state', 'state_symbol',
                         'month',]).agg({'overall_count' : 'sum', 'ev_count' : 'sum'}).reset_index()

merged_df3['ev_share'] = (merged_df3['ev_count']/merged_df3['overall_count'])*100

In [ ]:
registrations_dataset = merged_df3.copy()

In [ ]:
registrations_dataset

# Section 2: Creating shapefiles for grouped districts

In [ ]:
# READING THE SHAPEFILE CONTAINING BOUNDARIES FOR INDIVIDUAL DISTRICTS ACCORDING TO 2011 CENSUS

shapefiles = zf.ZipFile('2011 districts datameet boundaries.zip', 'r')
shapefiles.extractall('district shapefiles')
gdf_districts = gpd.read_file('district shapefiles/India-Districts-2011Census.shp')

state_symbol_dict = {'Andhra Pradesh' : 'AP', 'Uttar Pradesh' : 'UP', 'Gujarat' : 'GJ', 'Maharashtra' : 'MH',
                    'Mizoram' : 'MZ', 'Rajasthan' : 'RJ', 'Kerala' : 'KL', 'Madhya Pradesh' : 'MP',
                    'Uttarakhand' : 'UK', 'Haryana' : 'HR', 'Punjab' : 'PB', 'Jammu & Kashmir' : 'JK',
                    'Arunanchal Pradesh' : 'AR', 'Odisha' : 'OR', 'Bihar' : 'BR', 'Tamil Nadu' : 'TN',
                    'Karnataka' : 'KA', 'Assam' : 'AS', 'West Bengal' : 'WB', 'Chhattisgarh' : 'CG',
                     'Himachal Pradesh' : 'HP', 'Manipur' : 'MN', 'Jharkhand' : 'JH', 'NCT of Delhi' : 'DL',
                     'Chandigarh' : 'CH', 'Dadara & Nagar Havelli' : 'DD', 'Daman & Diu' : 'DD', 'Tripura' : 'TR',
                     'Nagaland' : 'NL', 'Sikkim' : 'SK', 'Meghalaya' : 'ML', 'Puducherry' : 'PY',
                     'Lakshadweep' : 'LD', 'Andaman & Nicobar Island' : 'AN', 'Goa' : 'GA'}

In [ ]:
gdf_districts

In [ ]:
# RECONCILING NAMING CONVENTIONS FOR SHAPEFILE MERGING LATER

gdf_districts['state_symbol'] = np.nan
for i in range(len(gdf_districts)):
    gdf_districts['state_symbol'][i] = state_symbol_dict[gdf_districts['ST_NM'][i]]

for i in range(len(gdf_districts)):
    gdf_districts['DISTRICT'][i] = gdf_districts['DISTRICT'][i] + ', ' + gdf_districts['state_symbol'][i]
    
df_for_gis_group_mapping = pd.read_excel('correct district naming.xlsx').drop(columns = 'Unnamed: 0').dropna()
gdf_districts = gdf_districts.merge(df_for_gis_group_mapping, on = 'DISTRICT')

In [ ]:
# THE GOAL IS TO CREATE BOUNDARIES FOR AMALGAMATED DISTRICTS USING THE BOUNDARIES FOR INDIVIDUAL DISTRICTS. FOR
# THAT, WE CREATE 2 SEPERATE SHAPEFILES: merge_set1 CONTAINS BOUNDARIES FOR THE FIRST DISTRICT AND merge_set2 
# CONTAINS BOUNDARIES FOR THE SECOND DISTRICT. IN CASES WHERE THE FIRST AND SECOND DISTRICT IS THE SAME, THE
# BOUNDARIES ARE ALSO THE SAME

merging_partner = merged_df1[['district_2011 (stage 2)', 'district_2011 (stage 2.1)',
                              'unique_districts', 'grouped_district']]
merging_partner['unique_districts'] = merging_partner['unique_districts'].apply(frozenset)
merging_partner = merging_partner.drop_duplicates()

merge_set1 = gdf_districts.merge(merging_partner, left_on = 'DISTRICT (FOR MERGING)',
                                 right_on = 'district_2011 (stage 2)', how = 'right')

merge_set1 = merge_set1[['DISTRICT (FOR MERGING)', 'state_symbol', 'geometry', 'district_2011 (stage 2)',
          'district_2011 (stage 2.1)', 'unique_districts', 'grouped_district']]

merge_set2 = gdf_districts.merge(merging_partner, left_on = 'DISTRICT (FOR MERGING)',
                                 right_on = 'district_2011 (stage 2.1)', how = 'right')

merge_set2 = merge_set2[['DISTRICT (FOR MERGING)', 'state_symbol', 'geometry', 'district_2011 (stage 2)',
          'district_2011 (stage 2.1)', 'unique_districts', 'grouped_district']]

merge_set3 = merge_set1.merge(merge_set2, on = ['grouped_district', 'unique_districts']).reset_index(drop = True)

merge_set4 = pd.DataFrame(columns = merge_set3.columns)
for i in range(len(merge_set3)):
    # EVEN IF THERE IS A ROW WITH AN ENTRY FOR AHMEDABAD, IT NEEDS TO BE MAPPED TO AHMEDABAD + BHAVNAGAR. WE FILTER
    # THESE CASES OUT. 
    if (len(merge_set3['unique_districts'][i]) != 2) & ('+' in merge_set3['grouped_district'][i]):
        pass
    else:
        merge_set4.loc[len(merge_set4)] = merge_set3.iloc[i]
        

# HERE, WE TAKE A UNION OF THE BOUNDARIES OF THE DIFFERENT DISTRICTS. IN CASE OF ONLY 1 SINGLE UNIQUE DISTRICT,
# THE UNION WOULD RETURN THE SAME BOUNDARY.
merge_set5 = merge_set4[['grouped_district', 'geometry_x', 'geometry_y', 'state_symbol_x']]
merge_set5['group_geometry'] = np.nan
for i in range(len(merge_set5)):
    poly1 = merge_set5['geometry_x'][i]
    poly2 = merge_set5['geometry_y'][i]
    poly = poly1.union(poly2)
    merge_set5['group_geometry'][i] = poly
    
merge_set5 = merge_set5.drop(columns = ['geometry_x', 'geometry_y'])
merge_set5 = merge_set5.rename(columns = {'state_symbol_x' : 'state_symbol'})

merge_set5 = gpd.GeoDataFrame(merge_set5, geometry = 'group_geometry', crs = merge_set1.crs)

merge_set5.to_file('grouped district boundaries.shp')

In [ ]:
grouped_district_boundaries = merge_set5.copy()

In [ ]:
grouped_district_boundaries

# Section 3: Meteorological variables

In [ ]:
# READING THE DIRECTORY CALLED weather data, APPENDING ALL THE FILES TOGETHER INTO A SINGLE LARGE DATAFRAME CALLED
# weather_df.
weather_files = os.listdir('weather data')
weather_files.sort()

weather_df = pd.read_excel('weather data/' + weather_files[0])
for i in range(1, len(weather_files)):
    to_add_weather_df = pd.read_excel('weather data/' + weather_files[i])
    weather_df = pd.concat([weather_df, to_add_weather_df])

In [ ]:
weather_df

In [ ]:
# weather_df CONTAINS COORDINATES AND NOT DISTRICTS. THEREFORE, WE MAP THESE COORDINATES TO THE GROUPED DISTRICTS
# WE CREATED IN THE PREVIOUS SECTION.

weather_df1 = weather_df.drop_duplicates().reset_index(drop = True)
group_gdf = gpd.read_file('latest grouped district boundaries.shp')

# maps the coordinates to districts using the intersects method
geometry = [Point(lon, lat) for lon, lat in zip(weather_df1['LON'], weather_df1['LAT'])]
gdf_weather = gpd.GeoDataFrame(weather_df1, geometry = geometry, crs = group_gdf.crs)
gdf_combined = gpd.sjoin(gdf_weather, group_gdf, how = 'right', op = 'intersects')

In [ ]:
gdf_combined

The resolution of the meteorological variables is very sparse, as the grid is comprised of 0.5 x 0.5 degree squares. The data is present at the corners of these 0.5 x 0.5 degree squares. Due to this sparseness, very small districts end up getting completely skipped out as it is possible that none of the corner points lie within a district's boundaries. In this data, about 90 districts end up getting skipped out. Our strategy to fill the missing data can be summarized in the following steps:
1. Establish which districts don't have weather data. In this data, we have 5 weather parameters (temperature, relative humidity, windspeed, dewpoint temperature, precipitation) and any district that doesn't have data for even one parameter doesn't have data for any other parameter either.
2. Create empty rows of such districts for each of the 5 parameters so that we can fill them up later on.
3. Create a column with the centroid of each of the districts with missing data.
4. Of the coordinates which have weather data, see which one is closest to the centroids of such districts (with missing data)
5. Fill the missing values with the values from that of the closest points.

In [ ]:
# STEPS 1,2, AND 3: ADDING EMPTY ROWS SO THAT WE CAN FILL THEM LATER ON USING NEAREST POINT IMPUTATION

# DROPPING ROWS WHICH AREN'T REQUIRED, # ANN is annual average for a particular parameter
gdf_combined = gdf_combined.drop(columns = ['index_left', 'ANN']) 
df2 = gdf_combined.copy()
df2['centroid'] = df2['geometry'].centroid

# adding the null rows corresponding to each parameter and year. 5 parameters, hence 5 different times.
to_concat = df2[df2['PARAMETER'].isnull()].copy()
to_concat['PARAMETER'] = 'T2M'
df2 = pd.concat([to_concat, df2],axis = 0)

to_concat = df2[df2['PARAMETER'].isnull()].copy()
to_concat['PARAMETER'] = 'RH2M'
df2 = pd.concat([to_concat, df2],axis = 0)

to_concat = df2[df2['PARAMETER'].isnull()].copy()
to_concat['PARAMETER'] = 'WS2M'
df2 = pd.concat([to_concat, df2],axis = 0)

to_concat = df2[df2['PARAMETER'].isnull()].copy()
to_concat['PARAMETER'] = 'T2MDEW'
df2 = pd.concat([to_concat, df2],axis = 0)

to_concat = df2[df2['PARAMETER'].isnull()].copy()
to_concat['PARAMETER'] = 'PRECTOTCORR'
df2 = pd.concat([to_concat, df2],axis = 0)

df2 = df2[df2['PARAMETER'].isnull() == False].copy() # skips out the rows which we used for replication purposes
# above

In [ ]:
# IN THE PREVIOUS CODE BLOCK, WE CREATE NEW ROWS BASED ON THE DISTRICTS 

months = df2.columns[4:16]  # gets the list of months
parameters = df2['PARAMETER'].unique()  # gets the list of parameters

new_df1 = df2[df2['YEAR'].isnull() == False].copy() # skips out the useless rows, which we use to create the
# required rows and then add to the main dataframe
new_df2 = df2[df2['YEAR'].isnull()].copy() 

years_range = pd.DataFrame({'YEAR': range(2010, 2023)})

to_add = pd.merge(years_range, new_df2, how='cross').drop(columns = ['YEAR_y'])

to_add = to_add[['PARAMETER', 'YEAR_x'] + list(to_add.columns[2:])].rename(columns = {'YEAR_x' : 'YEAR'})

In [ ]:
# SOME COLUMN RENAMING OPERATIONS AND THEN, ISOLATING THAT PART OF THE DATAFRAME WHICH NEEDS TO BE FILLED USING
# NEAREST POINT MATCHING

df3 = pd.concat([new_df1, to_add], axis = 0) # done
df3['YEAR'] = df3['YEAR'].astype(int)
df3 = df3.reset_index(drop = True)

# for districts which don't have points, we assign them the centroid's coordinates, and proceed thereafter.
df3['LAT'] = df3['LAT'].fillna(df3['centroid'].apply(lambda point: point.y))
df3['LON'] = df3['LON'].fillna(df3['centroid'].apply(lambda point: point.x))

df3 = df3.rename(columns = {'state_symb' : 'state', 'grouped_di' : 'district'})
df3 = pd.melt(df3, id_vars = ['district', 'state', 'geometry', 'LAT', 'LON', 'centroid', 'PARAMETER', 'YEAR'],
       var_name = 'month', value_name = 'measure')

new_df = df3.drop(columns = ['geometry', 'centroid']).copy()
newer_df = new_df.pivot(index = ['district', 'state', 'LAT', 'LON', 'YEAR', 'month'], columns = 'PARAMETER',
            values = 'measure').reset_index()
newer_df.columns.name = None
filling_df = newer_df.copy()
filled_df = filling_df.copy()

nulls_df = filled_df[filled_df['T2M'].isnull()] # seperate dataframe of nulls which we will populate with
# imputed values
nulls_df = nulls_df.reset_index(drop = True)
filled_df = filled_df[filled_df['T2M'].isna() == False]
full_frame = filling_df[['LAT', 'LON']].drop_duplicates().reset_index(drop = True)

In [ ]:
# STEPS 4 AND 5
coordinates = full_frame[['LAT', 'LON']].values

# Create a Nearest Neighbors model
k = 2  # Number of neighbors to consider (adjust as needed)
nn_model = NearestNeighbors(n_neighbors=k)
nn_model.fit(coordinates)

search_frame = nulls_df[['LAT', 'LON']].drop_duplicates().reset_index(drop = True)
match_values = filled_df[['LAT', 'LON']].drop_duplicates().reset_index(drop = True).values

# creating the frames to match on
search_frame['LAT_matched'] = np.nan
search_frame['LON_matched'] = np.nan

for i in range(len(search_frame)):
    to_search = nulls_df[['LAT', 'LON']].drop_duplicates().reset_index(drop = True).values[i]
    _, indices = nn_model.kneighbors(to_search.reshape(1,-1))
    for j in indices[0]:
        if full_frame.loc[j].values in match_values:
            matched_coord_index = j
            break
        else:
            pass
    search_frame['LAT_matched'][i] = full_frame.loc[matched_coord_index]['LAT']
    search_frame['LON_matched'][i] = full_frame.loc[matched_coord_index]['LON']
    
matched_nulls_df = nulls_df.merge(search_frame, on = ['LAT', 'LON'])
matched_nulls_df = matched_nulls_df.rename(columns = {'LAT' : 'LAT_tomatch', 'LON' : 'LON_tomatch',
                                                     'LAT_matched' : 'LAT', 'LON_matched' : 'LON'})

In [ ]:
# AFTER RUNNING THE NEAREST NEIGHBOR ALGORITHM, LAT_matched and LON_matched are the points (for which we have data)
# which are the closest to the districts' (with missing data) centroids
search_frame

In [ ]:
matched_nulls_df

In [ ]:
# FURTHER FORMATTING AND RENAMING COLUMNS AND VALUES FOR MERGING LATER WITH THE OTHER (VEHICLE REGISTRATIONS,
# PM2.5, NIGHTLIGHT DATA) DATAFRAMES
matched_nulls_df2 = matched_nulls_df.merge(filled_df, on = ['LAT', 'LON', 'YEAR', 'month'], how = 'inner')

for i in range(len(matched_nulls_df2)):
    matched_nulls_df2['PRECTOTCORR_x'][i] = matched_nulls_df2['PRECTOTCORR_y'][i]
    matched_nulls_df2['RH2M_x'][i] = matched_nulls_df2['RH2M_y'][i]
    matched_nulls_df2['T2M_x'][i] = matched_nulls_df2['T2M_y'][i]
    matched_nulls_df2['T2MDEW_x'][i] = matched_nulls_df2['T2MDEW_y'][i]
    matched_nulls_df2['WS2M_x'][i] = matched_nulls_df2['WS2M_y'][i]

matched_nulls_df2 = matched_nulls_df2[['district_x', 'state_x', 'LAT_tomatch', 'LON_tomatch', 'YEAR', 'month', 
         'PRECTOTCORR_x', 'RH2M_x', 'T2M_x', 'T2MDEW_x', 'WS2M_x']]
matched_nulls_df2 = matched_nulls_df2.rename(columns = {'district_x' : 'district', 'state_x' : 'state', 'LAT_tomatch' : 'LAT',
                                     'LON_tomatch' : 'LON', 'PRECTOTCORR_x' : 'PRECTOTCORR', 'RH2M_x' : 'RH2M',
                                     'T2M_x' : 'T2M', 'T2MDEW_x' : 'T2MDEW', 'WS2M_x' : 'WS2M'})

updated_filled_df = pd.concat([filled_df, matched_nulls_df2]).reset_index(drop = True)

month_mapping = {'JAN' : 1, 'FEB' : 2, 'MAR' : 3, 'APR' : 4, 'MAY' : 5, 'JUN' : 6, 'JUL' : 7,
              'AUG' : 8, 'SEP' : 9, 'OCT' : 10, 'NOV' : 11, 'DEC' : 12}

In [ ]:
# Function to map month initials to month numbers
def map_month_initials(month_initial):
    return month_mapping.get(month_initial.upper(), None)

# Apply the mapping function to create a new column with month numbers
updated_filled_df['month'] = updated_filled_df['month'].apply(map_month_initials)

In [ ]:
updated_filled_df['YEAR'] = updated_filled_df['YEAR'].astype(str)
updated_filled_df['month'] = updated_filled_df['month'].astype(str)
updated_filled_df['year_month'] = updated_filled_df['YEAR'] + '-' + updated_filled_df['month']
updated_filled_df['year_month'] = pd.to_datetime(updated_filled_df['year_month'])
updated_filled_df = updated_filled_df.drop(columns = ['YEAR', 'month'])

updated_filled_df = updated_filled_df.groupby(by = ['district', 'state', 'year_month']).agg({'PRECTOTCORR' : 'mean',
                                                                       'RH2M' : 'mean',
                                                                       'T2M' : 'mean',
                                                                       'T2MDEW' : 'mean',
                                                                       'WS2M' : 'mean'}).reset_index()

In [ ]:
weather_dataset = updated_filled_df.copy()

In [ ]:
weather_dataset

# Section 4: Urban PM2.5

Electric vehicles are specifically promoted for urban areas, and therefore, it makes sense to measure its impact on urban pollution specifically. NASA classified localities in India into 5 classes of urbanness, based on GHSL. These classes are:
1. BUILT UP LAND ONLY
2. RURAL
3. UNINHABITED
4. AGGREMENT
5. URBAN PEOPLE ONLY

To isolate urban areas out of these 5 classes, we focus only on 1,4, and 5.

In [ ]:
urban_files = zf.ZipFile('India urbanness shapefile Columbia.zip', 'r')
urban_files.extractall('India urbanness shapefiles')

file_location = 'India urbanness shapefiles/india-spatial-india-census-2011-census-ghsl-50pct-shp/india-spatial-india-census-2011_census-ghsl-50pct-india.shp'
gdf_urban = gpd.read_file(file_location)
gdf_urban

In [ ]:
rows_to_drop = [1, 2]

# dropping the rural and uninhabited columns as they are redundant once we have the urban areas, and also
# we only require their PM2.5 levels for the pollution measure.
gdf_urban = gdf_urban.drop(index = rows_to_drop)

urban1 = gpd.overlay(grouped_district_boundaries, gdf_urban.tail(2).head(1), how = 'intersection')
urban1.to_file('urban1.shp')

urban2 = gpd.overlay(grouped_district_boundaries, gdf_urban.tail(1), how = 'intersection')
urban2.to_file('urban2.shp')

urban3 = gpd.overlay(grouped_district_boundaries, gdf_urban.head(1), how = 'intersection')
urban3.to_file('urban3.shp')


In [ ]:
all_urbans = gpd.GeoDataFrame(pd.concat([urban1, urban2, urban3], ignore_index = True, axis = 0))

all_urbans = all_urbans.groupby(['grouped_district', 'state_symbol'])['geometry'].apply(lambda x: x.unary_union).reset_index()

all_urbans.to_file('district urban area shapefiles.shp')

In [ ]:
all_urbans

In [ ]:
# THE PM2.5 DATA FILES FROM WUSTL CONTAIN DATA FOR ALL COORDINATES WITHIN ASIA. THIS CODE BLOCK ISOLATES THAT TO
# JUST COORDINATES BELONGING TO INDIA. GIVEN THAT THE FILES CONTAINS MILLIONS OF POINTS, IT BECOMES IMPORTANT TO
# ISOLATE IT JUST TO INDIA FOR COMPUTATIONAL SPEED. ALL THE MONTHLY FILES ARE APPENDED TO THE LIST overall_pm25

directory = 'higher resolution PM2.5 data monthly files/'
files_list = os.listdir(directory)
files_list.sort()

pollution_df_file = files_list[27] # starting from april 2012

data = Dataset(directory + pollution_df_file)

lon_data = data.variables['lon'][:]
lat_data = data.variables['lat'][:]
pm25_data = data.variables['GWRPM25']

lon_data = [ round(lon, 4) for lon in lon_data ]
lat_data = [ round(lat, 4) for lat in lat_data ]

india_lat = []
india_lon = []
for lon in lon_data:
    if int(lon) in range(68,98):
        india_lon.append(lon)

for lat in lat_data:
    if int(lat) in range(8, 38):
        india_lat.append(lat)

pollution_df = xr.open_dataset(directory + pollution_df_file)
pollution_df = pollution_df.to_dataframe()
pollution_df = pollution_df.reset_index()
pollution_df['lon'] = pollution_df['lon'].round(4)
pollution_df['lat'] = pollution_df['lat'].round(4)
pollution_df = pollution_df[(pollution_df['lat'] >= india_lat[0]) & (pollution_df['lat'] <= india_lat[-1])
                & (pollution_df['lon'] >= india_lon[0]) & (pollution_df['lon'] <= india_lon[-1])]

time_period = pollution_df_file.split('.')[3].split('-')[0]
pollution_df = pollution_df.rename(columns = {'GWRPM25' : time_period})

overall_pm25 = [pollution_df]

for file in files_list[28:]: # continuing from may 2012
    month_pollution_df = xr.open_dataset(directory + file)
    month_pollution_df = month_pollution_df.to_dataframe()
    month_pollution_df = month_pollution_df.reset_index()
    month_pollution_df['lon'] = month_pollution_df['lon'].round(4)
    month_pollution_df['lat'] = month_pollution_df['lat'].round(4)
    month_pollution_df = month_pollution_df[(month_pollution_df['lat'] >= india_lat[0]) & (month_pollution_df['lat'] <= india_lat[-1])
                    & (month_pollution_df['lon'] >= india_lon[0]) & (month_pollution_df['lon'] <= india_lon[-1])]
    
    time_period = file.split('.')[3].split('-')[0]
    month_pollution_df = month_pollution_df.rename(columns = {'lon' : 'lon_' + time_period, 'lat' : 'lat_' + time_period,
                           'GWRPM25' : time_period})
    overall_pm25.append(month_pollution_df[time_period])
    
    #print(str(files_list.index(file)) + ' ' + time_period + ' done')
    

In [ ]:
# VERTICALLY APPENDING ALL THE FILES PRESENT IN overall_pm25
merged_pollution_df = pd.concat(overall_pm25, axis = 1)

In [ ]:
# MAPPING THE PM2.5 DATA (WHICH IS FOR COORDINATES AND NOT DISTRICTS DIRECTLY) TO THE GROUPED DISTRICT SHAPEFILES
spatial_df = merged_pollution_df.iloc[:, :3]
spatial_df['lat'] = spatial_df['lat'].astype(float)
spatial_df['lon'] = spatial_df['lon'].astype(float)

geometry = [Point(lon, lat) for lon, lat in zip(spatial_df['lon'], spatial_df['lat'])]
print('geometry created')
gdf_ntl = gpd.GeoDataFrame(spatial_df, geometry = geometry, crs = all_urbans.crs)
print('dataframe created')
gdf_combined = gpd.sjoin(gdf_ntl, all_urbans, how = 'inner', predicate = 'intersects')
print('sjoin done')

gdf_locations = gdf_combined.copy() # gives you the location reference for each point in the cities
urbans_df = merged_pollution_df.loc[gdf_locations.index]
gdf_locations = gdf_locations.dropna()[['lon', 'lat', 'district']]
urbans_df['lat'] = urbans_df['lat'].astype(float)
urbans_df['lon'] = urbans_df['lon'].astype(float)
urbans_df = urbans_df.merge(gdf_locations, on = ['lon', 'lat'])

urbans_data = pd.melt(urbans_df, id_vars=['lon',
                                          'lat', 'district'], var_name = 'year_month', value_name = 'GWRPM25')
urbans_data = urbans_data[['district', 'year_month', 'GWRPM25']]
urbans_data['year_month'] = urbans_data['year_month'].apply(lambda x: str(x[:4]) + '-' + str(x[4:]))
urbans_data['year_month'] = pd.to_datetime(urbans_data['year_month'])
urbans_data = urbans_data.groupby(by = ['district', 'year_month'])['GWRPM25'].mean().to_frame().reset_index()

In [ ]:
urban_pm25_dataset = urbans_data.copy()

In [ ]:
urban_pm25_dataset

# Section 5: Urban nightlight

When downloaded, the nightlight images are 2GB in size (for each month). to reduce this to a manageable size, we write the below nightlight_data function to accomplish two things mainly:
1. To restrict the image's data to India's boundaries. While the 2GB files contain information on all of Asia, this is unnecessary.
2. To upscale/downscale the resolution of the pictures. When I upload the images on dropbox, this function helps me reduce the 2GB images to roughly 3 MB each. However, at that level, the resolution is too low to have enough data to map to the districts. Therefore, once the image has been downloaded from dropbox to your system, the below function upscales these images by a factor of 10 (on each dimension) to arrive at 300MB image files. This level is ideal for the level of my units (i.e. districts).

In [ ]:
def find_closest_element(lst, target):
    return min(lst, key = lambda x: abs(x - target))

def nightlight_data(tif_file_path, upscale_factor):
    
    with rasterio.open(tif_file_path) as dataset:
        # Resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * upscale_factor),
                int(dataset.width * upscale_factor)
            ),
            resampling=Resampling.average
        )

        # Scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )

        # Create a new GeoTIFF file for the resampled data
        output_file = tif_file_path
        with rasterio.open(output_file, 'w', driver='GTiff', height=data.shape[-2], width=data.shape[-1], count=dataset.count, dtype=data.dtype, crs=dataset.crs, transform=transform) as dst:
            dst.write(data)

    tif_file_path = output_file
    dataarray = rxr.open_rasterio(tif_file_path)
    dataarray = xr.open_rasterio(tif_file_path)

    ntl_df = dataarray[0].to_pandas()

    lonlist = list(ntl_df.columns)
    latlist = list(ntl_df.index)

    lon_lower = str(find_closest_element(lonlist, 68))
    lon_upper = str(find_closest_element(lonlist, 98))

    lat_lower = str(find_closest_element(latlist, 8))
    lat_upper = str(find_closest_element(latlist, 38))

    ntl_df.columns = ntl_df.columns.astype(str)
    ntl_df.index = ntl_df.index.astype(str)

    lon_lower_index = ntl_df.columns.get_loc(lon_lower)
    lon_upper_index = ntl_df.columns.get_loc(lon_upper)

    lat_lower_index = ntl_df.index.get_loc(lat_lower)
    lat_upper_index = ntl_df.index.get_loc(lat_upper)

    ntl_df = ntl_df.iloc[:, lon_lower_index : lon_upper_index + 1]
    ntl_df = ntl_df.iloc[lat_upper_index : lat_lower_index + 1, :]

    exp_ntl_df = ntl_df.reset_index()
    exp_ntl_df = exp_ntl_df.rename(columns = {'y' : 'lat'})
    to_melt_list = list(exp_ntl_df.columns[1:])
    exp_ntl_df = pd.melt(exp_ntl_df, id_vars = ['lat'], value_vars = to_melt_list).rename(columns = {'x' : 'lon',
                                                                                            'value' : 'nightlight'})
    
    year_month = tif_file_path.split('-')[0].split('_')[-1][:6]
    exp_ntl_df = exp_ntl_df.rename(columns = {'lat' : 'lat_' + year_month,
                                      'lon' : 'lon_' + year_month,
                                     'nightlight' : year_month})
    #exp_ntl_df.to_csv('nightlight_data - ' + exp_ntl_df['year_month'][0] + '.csv')
    return (exp_ntl_df)

In [ ]:
# THE CODE READS AND APPENDS EACH MONTH'S NIGHTLIGHT DATA TO NTL_DF_LIST. COMBINED_DF APPENDS ALL THE NIGHTLIGHT
# DATA FROM NTL_DF_LIST

files_list = os.listdir('Monthly nightlight data (all years)')
files_list.sort()

df_file = files_list[0] # starting from april 2012
directory = 'Monthly nightlight data (all years)/'

main_df_file = directory + df_file
main_df = nightlight_data(main_df_file, 10)

print(main_df.columns[2] + ' done')

ntl_df_list = [main_df]
for i in files_list[1:]:
    month_df_file = directory + i
    data_column = i.split('-')[0].split('_')[-1][:6]
    month_df = nightlight_data(month_df_file, 10) 
    ntl_df_list.append(month_df[data_column])
    print(data_column + ' done')

combined_df = pd.concat(ntl_df_list, axis = 1)

In [ ]:
# MAPS THE COORDINATES TO THE DISTRICTS, AS IN THE PREVIOUS CASE.
spatial_df = combined_df.iloc[:, :3]
spatial_df = spatial_df.rename(columns = {'lat_202208' : 'lat', 'lon_202208' : 'lon'})
spatial_df['lat'] = spatial_df['lat'].astype(float)
spatial_df['lon'] = spatial_df['lon'].astype(float)

geometry = [Point(lon, lat) for lon, lat in zip(spatial_df['lon'], spatial_df['lat'])]
print('geometry created')
gdf_ntl = gpd.GeoDataFrame(spatial_df, geometry = geometry, crs = all_urbans.crs)
print('dataframe created')
gdf_combined = gpd.sjoin(gdf_ntl, all_urbans, how = 'inner', predicate = 'intersects')
print('sjoin done')

gdf_locations = gdf_combined.copy() # gives you the location reference for each point in the cities
urban_ntl_df = combined_df.loc[gdf_locations.index]
gdf_locations = gdf_locations.dropna()[['lon', 'lat', 'district']]
urban_ntl_df = urban_ntl_df.rename(columns = {'lat_202208' : 'lat', 'lon_202208' : 'lon'})
urban_ntl_df['lat'] = urban_ntl_df['lat'].astype(float)
urban_ntl_df['lon'] = urban_ntl_df['lon'].astype(float)
urban_ntl_df = urban_ntl_df.merge(gdf_locations, on = ['lon', 'lat'])

urban_ntl_data = pd.melt(urban_ntl_df, id_vars=['lon',
                                          'lat', 'district'], var_name = 'year_month', value_name = 'nightlight')
urban_ntl_data = urban_ntl_data[['district', 'year_month', 'nightlight']]
urban_ntl_data['year_month'] = urban_ntl_data['year_month'].apply(lambda x: str(x[:4]) + '-' + str(x[4:]))
urban_ntl_data['year_month'] = pd.to_datetime(urban_ntl_data['year_month'])
urban_ntl_data = urban_ntl_data.groupby(by = ['district', 'year_month'])['nightlight'].mean().to_frame().reset_index()

In [ ]:
urban_ntl_dataset = urban_ntl_data.copy()

In [ ]:
urban_ntl_dataset

# Merging the created datasets

Based on grouped_district_boundaries, we merge the following 4 datasets:
1. registrations_dataset
2. weather_dataset
3. urban_pm25_dataset
4. urban_ntl_dataset

We merge the attributes from each of the datasets on the district and year-month column. Since the naming of attributes is different only in registrations_dataset and same in the others, we make the following modifications to registrations_dataset:
1. rename grouped_district to district
2. bring year and month together into a column called year_month, with the same format as the other datasets (using month_mapping, created towards the end of section 3)

In [ ]:
registrations_dataset

In [ ]:
weather_dataset

In [ ]:
urban_pm25_dataset

In [ ]:
urban_ntl_dataset

In [ ]:
registrations_dataset = registrations_dataset.rename(columns = {'grouped_district' : 'district'})
registrations_dataset['month'] = registrations_dataset['month'].apply(lambda x: month_mapping[x])
registrations_dataset['year_month'] = registrations_dataset['year'].astype(str) + '-' + registrations_dataset['month'].astype(str)
registrations_dataset['year_month'] = pd.to_datetime(registrations_dataset['year_month'])

In [ ]:
master_dataset = pd.merge(registrations_dataset, weather_dataset,
                          on = ['district', 'year_month'])

In [ ]:
master_dataset = pd.merge(master_dataset, urban_pm25_dataset,
                          on = ['district', 'year_month'])

In [ ]:
master_dataset = pd.merge(master_dataset, urban_ntl_dataset,
                          on = ['district', 'year_month'])

In [ ]:
master_dataset = master_dataset.drop(columns = ['state_x', 'state_y', 'year', 'month'])

In [ ]:
master_dataset.to_csv('Dataset for EV and pollution paper.csv')